In [1]:
import os
import csv
import pandas as pd
import time
import pickle
import sys
import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from collections import defaultdict


In [2]:
print(os.getcwd())
# it should end with this: /AITutor_SeqModeling
# if not, run the next block


/local-scratch/localhome/pagand/projects/mygitsDaTu/AITutor_SeqModeling/KnowledgeTracking


In [3]:
# run if the current directory is not AITutor_SeqModeling
cwd = os.chdir(os.path.join(os.getcwd(), ".."))
print(os.getcwd())

/local-scratch/localhome/pagand/projects/mygitsDaTu/AITutor_SeqModeling


In [4]:
File_pickle = "data/KT_logs_annotated.pkl"

# read from pickle
df = pd.read_pickle(File_pickle)

df.head()

,username,skill,correct,time
0,a1,"[Supervised Learning, Classification Algorithms]",True,0.000000
1,a1,"[Supervised Learning, Classification Algorithms]",False,3.726700
2,a2,"[Supervised Learning, Classification Algorithms]",False,0.000000
3,a2,"[Supervised Learning, Classification Algorithms]",True,1.987467
4,a3,"[Supervised Learning, Classification Algorithms]",True,0.000000


In [9]:
skills = pickle.load(open("data/Skill_hirereachy.pkl", "rb"))

In [ ]:
# version 1: Only user params
# user_params = {}

# def initialize_user_params(user_id, skills):
#     user_params[user_id] = {}
#     for skill in skills.keys():
#         skill_params = skills[skill][-1]
#         user_params[user_id][skill] = {
#             "P(L)": skill_params[0],
#             "P(T)": skill_params[1],
#             "P(G)": skill_params[2],
#             "P(S)": skill_params[3]
#         }
# for user_id in df["username"].unique():
#     initialize_user_params(user_id, skills)

In [10]:
# version 2: seperate user-specific and skill specific parameters
def initialize_params(skills, user_ids):
    user_params = {}
    skill_params = {}
    for skill in skills.keys():
        skill_params[skill] = {
                "P(L)": skills[skill][-1][0],
                "P(T)": skills[skill][-1][1],
                "P(G)": skills[skill][-1][1],
                "P(S)": skills[skill][-1][3]
            }
        for user_id in user_ids:
            # assume all users have the same initial skill level
            # add prior knowledge here if exists
            user_params[user_id] = skill_params.copy()
            user_params[user_id]['weight'] = 0.5
    return skill_params, user_params

In [11]:
skill_params, user_params = initialize_params(skills,  df["username"].unique())
user_params['a1']["weight"]

0.5

In [12]:
# Function to compute P(C_t|L_t, G, S)
def compute_prob_correctness(P_L, P_G, P_S, P_T, correct):
    if correct:
        P_L_obs = P_L* (1 - P_S) /((1 - P_S) * P_L + P_G * (1 - P_L))
    else:
        P_L_obs = P_L* (P_S) /(P_S * P_L + (1 - P_G) * (1 - P_L))
    
    P_L_new = P_L_obs + (1 - P_L_obs) * P_T
    P_C = P_L_new * (1 - P_S) + (1 - P_L_new) * P_G

    # debug
    if not (0 <= P_C <= 1):
        print(f"Invalid P(C): {P_C}, P_L: {P_L}, P_G: {P_G}, P_S: {P_S}, P_T: {P_T}")
    return P_C, P_L_new


In [13]:
# (log-likelihood function)
def log_likelihood(interaction_log,  skill_params, user_params):
    log_likelihood = 0
    expectations = []
    for _, row in interaction_log.iterrows():
        user_id = row["username"]
        skill_list = row["skill"]
        correctness = row["correct"]

        for skill in skill_list:
            # Retrieve user and skill parameters
            P_L_user = user_params[user_id][skill]["P(L)"]
            P_G_user = user_params[user_id][skill]["P(G)"]
            P_S_user = user_params[user_id][skill]["P(S)"]
            P_T_user = user_params[user_id][skill]["P(T)"]
            P_user = [P_L_user, P_G_user, P_S_user, P_T_user]

            P_L_skill = skill_params[skill]["P(L)"]
            P_G_skill = skill_params[skill]["P(G)"]
            P_S_skill = skill_params[skill]["P(S)"]
            P_T_skill = skill_params[skill]["P(T)"]
            P_skill = [P_L_skill, P_G_skill, P_S_skill, P_T_skill]
            
            weight = user_params[user_id]['weight']

            # Weighted average for P(L)
            P_L = weight * P_L_user + (1-weight) * P_L_skill
            P_G = weight * P_G_user + (1-weight) * P_G_skill
            P_S = weight * P_S_user + (1-weight) * P_S_skill
            P_T = weight * P_T_user + (1-weight) * P_T_skill

            # Compute likelihood
            prob, _ = compute_prob_correctness(P_L, P_G, P_S, P_T, correctness)
            log_likelihood += np.log(prob + 1e-9)
            expectations.append((user_id, skill, prob, correctness, P_user, P_skill))

    return -log_likelihood, expectations

In [36]:
# E-Step: Calculate expected probabilities
def expectation_step(interaction_log, skill_params, user_params):
    skill_grads = defaultdict(lambda: {"P(L)": 0, "P(T)": 0, "P(G)": 0, "P(S)": 0})
    user_grads = defaultdict(lambda: defaultdict(lambda: {"P(L)": 0, "P(T)": 0, "P(G)": 0, "P(S)": 0}))
    weight_grads = {user_id: 0 for user_id in user_params.keys()}
    loss = 0
    likelihood, expectations = log_likelihood(interaction_log,  skill_params, user_params)
    if np.isnan(likelihood):
        print("Log-likelihood NaN detected! Check parameter updates.")

    for user_id, skill, prob, correctness, P_user, P_skill in expectations:
        grad = (correctness - prob) / (prob + 1e-9)
        loss += (correctness - prob) ** 2

        # make each value in user_grads[user_id][skill] 0.5 of its original value

        # Update user-specific gradients
        w = user_params[user_id]['weight']
        user_grads[user_id][skill]["P(L)"] = 0.5*(user_grads[user_id][skill]["P(L)"]  +
                                            grad * w * ((1 - P_user[2])- P_user[1]))
        user_grads[user_id][skill]["P(G)"] += 0.5*(user_grads[user_id][skill]["P(G)"] +
                                            grad * w *(1 - P_user[0]))
        user_grads[user_id][skill]["P(S)"] += 0.5*(user_grads[user_id][skill]["P(S)"] +
                                            grad* w *(-P_user[0]))
        user_grads[user_id][skill]["P(T)"] += 0.5*(user_grads[user_id][skill]["P(T)"] +
                                            grad* w * (1 - P_user[0])*((1 - P_user[2])- P_user[1]))

        weight_grads[user_id] = 0.5*(weight_grads[user_id]+ grad)

        # Update skill-specific gradients (aggregated across users)
        w = -w +1
        skill_grads[skill]["P(L)"] = 0.5* (skill_grads[skill]["P(L)"]+ grad * w * ((1 - P_skill[2])- P_skill[1]))
        skill_grads[skill]["P(G)"] = 0.5* (skill_grads[skill]["P(G)"]+ grad * w *(1 - P_skill[0]))
        skill_grads[skill]["P(S)"] = 0.5*(skill_grads[skill]["P(S)"] + grad* w *(-P_skill[0]))
        skill_grads[skill]["P(T)"] = 0.5*(skill_grads[skill]["P(T)"] + grad* w * (1 - P_skill[0])*((1 - P_skill[2])- P_skill[1]))


    return skill_grads, user_grads, weight_grads, likelihood, loss

In [37]:
# M-Step: Update parameters
def maximization_step(skill_params, user_params, skill_grads, user_grads, weight_grads, learning_rate=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8, t=1):
    # Adam accumulators
    skill_m = defaultdict(lambda: {"P(L)": 0, "P(T)": 0, "P(G)": 0, "P(S)": 0})
    skill_v = defaultdict(lambda: {"P(L)": 0, "P(T)": 0, "P(G)": 0, "P(S)": 0})
    user_m = defaultdict(lambda: defaultdict(lambda: {"P(L)": 0, "P(T)": 0, "P(G)": 0, "P(S)": 0}))
    user_v = defaultdict(lambda: defaultdict(lambda: {"P(L)": 0, "P(T)": 0, "P(G)": 0, "P(S)": 0}))


    # Apply SGD for user-specific updates
    for user_id, skills in user_grads.items():
        for skill, grads in skills.items():
            for param, grad in grads.items():
                    m = user_m[user_id][skill][param]
                    v = user_v[user_id][skill][param]

                    m = beta1 * m + (1 - beta1) * grad
                    v = beta2 * v + (1 - beta2) * (grad ** 2)
                    m_hat = m / (1 - beta1 ** t)
                    v_hat = v / (1 - beta2 ** t)

                    user_params[user_id][skill][param] = np.clip(user_params[user_id][skill][param] - learning_rate * m_hat / (np.sqrt(v_hat) + epsilon), 0, 1)
                    user_m[user_id][skill][param] = m
                    user_v[user_id][skill][param] = v

    # Apply batch updates for skill-specific parameters
    for skill, grads in skill_grads.items():
        for param, grad in grads.items():
            m = skill_m[skill][param]
            v = skill_v[skill][param]

            m = beta1 * m + (1 - beta1) * grad
            v = beta2 * v + (1 - beta2) * (grad ** 2)
            m_hat = m / (1 - beta1 ** t)
            v_hat = v / (1 - beta2 ** t)

            skill_params[skill][param] = np.clip(skill_params[skill][param] - learning_rate * m_hat / (np.sqrt(v_hat) + epsilon), 0, 1)
            skill_m[skill][param] = m
            skill_v[skill][param] = v
            
    # Update user weights
    for user_id in user_params.keys():
        user_params[user_id]['weight'] = np.clip(user_params[user_id]['weight'] - learning_rate * weight_grads[user_id], 0, 1)

    return skill_params, user_params

In [38]:
# Parent-Child Constraints
def enforce_constraints(user_params, skill_params, skills):
    for skill, skill_data in skills.items():
        parents = skill_data[1]
        for parent in parents:
            if skill_params[skill]["P(L)"] >= skill_params[parent]["P(L)"]:
                skill_params[skill]["P(L)"] = skill_params[parent]["P(L)"] - 0.01 
            for user in user_params:
                parent_prob = user_params[user][parent]["P(L)"]
                child_prob = user_params[user][skill]["P(L)"]
                if child_prob >= parent_prob:
                    user_params[user][skill]["P(L)"] = parent_prob - 0.01  # Apply heuristic

In [39]:
# Projection
def project_params(params, par_type):
    for key, param_set in params.items():
        if  par_type == "user":
                for skill, ps in param_set.items():
                    if skill == 'weight':
                        params[key][skill] = np.clip(params[key][skill], 0, 1)
                    else:
                        for param in ps:
                            params[key][skill][param] = np.clip(params[key][skill][param], 0, 1)
        elif par_type == "skill":
            if type(param_set) != dict:
                a = 3
            for param in param_set:
                params[key][param] = np.clip(params[key][param], 0, 1)
        else:
            raise ValueError("Invalid parameter type. Must be 'user' or 'skill'.")
            
    return params

In [40]:
def run_em_with_sgd(interaction_log, skills, max_iter=50, learning_rate=0.005):
    skill_params, user_params = initialize_params(skills, interaction_log["username"].unique())

    for i in range(max_iter):
        skill_grads, user_grads, weight_grads, likelihood, loss = expectation_step(interaction_log, skill_params, user_params)
        skill_params, user_params = maximization_step(skill_params, user_params, skill_grads, user_grads, weight_grads,learning_rate=learning_rate)
        skill_params = project_params(skill_params, "skill")
        user_params = project_params(user_params, "user")
        if i % 5 == 0:
            print(f"Iteration {i + 1}, Log-likelihood: {likelihood}, Loss: {loss}")

    return skill_params, user_params

In [41]:
# Run the EM Algorithm
skill_params, user_params = run_em_with_sgd(df.iloc[:1000], skills)

# Output Results
print("Final Skill Parameters:")
for skill, params in skill_params.items():
    print(skill, params)

print("\nFinal User Parameters:")
for user_id, user_data in user_params.items():
    print(user_id)
    for skill, params in user_data.items():
        print(f"  {skill}: {params}")

Iteration 1, Log-likelihood: 1603.322112971452, Loss: 210.05448063767454


/tmp/ipykernel_3801076/1216865270.py:6: RuntimeWarning: invalid value encountered in scalar divide
  P_L_obs = P_L* (P_S) /(P_S * P_L + (1 - P_G) * (1 - P_L))
/tmp/ipykernel_3801076/1216865270.py:4: RuntimeWarning: invalid value encountered in scalar divide
  P_L_obs = P_L* (1 - P_S) /((1 - P_S) * P_L + P_G * (1 - P_L))


Invalid P(C): nan, P_L: 1.0, P_G: 1.0, P_S: 0.0, P_T: 1.0
Invalid P(C): nan, P_L: 1.0, P_G: 1.0, P_S: 0.0, P_T: 1.0
Invalid P(C): nan, P_L: 1.0, P_G: 1.0, P_S: 0.0, P_T: 1.0
Invalid P(C): nan, P_L: 1.0, P_G: 1.0, P_S: 0.0, P_T: 1.0
Invalid P(C): nan, P_L: 1.0, P_G: 1.0, P_S: 0.0, P_T: 1.0
Invalid P(C): nan, P_L: 1.0, P_G: 1.0, P_S: 0.0, P_T: 1.0
Invalid P(C): nan, P_L: 1.0, P_G: 1.0, P_S: 0.0, P_T: 1.0
Invalid P(C): nan, P_L: 1.0, P_G: 1.0, P_S: 0.0, P_T: 1.0
Invalid P(C): nan, P_L: 1.0, P_G: 1.0, P_S: 0.0, P_T: 1.0
Invalid P(C): nan, P_L: 1.0, P_G: 1.0, P_S: 0.0, P_T: 1.0
Invalid P(C): nan, P_L: 1.0, P_G: 1.0, P_S: 0.0, P_T: 1.0
Invalid P(C): nan, P_L: 1.0, P_G: 1.0, P_S: 0.0, P_T: 1.0
Invalid P(C): nan, P_L: 1.0, P_G: 1.0, P_S: 0.0, P_T: 1.0
Invalid P(C): nan, P_L: 1.0, P_G: 1.0, P_S: 0.0, P_T: 1.0
Invalid P(C): nan, P_L: 1.0, P_G: 1.0, P_S: 0.0, P_T: 1.0
Invalid P(C): nan, P_L: 1.0, P_G: 1.0, P_S: 0.0, P_T: 1.0
Invalid P(C): nan, P_L: 1.0, P_G: 1.0, P_S: 0.0, P_T: 1.0
Invalid P(C): 

KeyboardInterrupt: 